In [68]:
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common import action_chains, keys
# from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# from pyvirtualdisplay import Display
# import gnp
# import codecs
# import json
# from boto.s3.connection import S3Connection
# from boto.s3.key import Key
# import boto
import sys, os
import pandas as pd
# from TwitterSearch import *
# from sunlight import congress
# from bs4 import BeautifulSoup
# import urllib2
# import itertools
# import datetime
# import configparser
# import time
from pattern.en import parsetree
from pattern.en import mood
import dateutil.parser as dparser
import sunlight
from sunlight import congress
import re
from postal.parser import parse_address
import csv
import selenium
from bs4 import BeautifulSoup as soupy
import tweepy
import codecs
import io
import phonenumbers
import re
import editdistance
import nltk
from geopy.geocoders import Nominatim
import address
from address import AddressParser, Address
import geotext
from geotext import GeoText
import geograpy
import pprint

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import editdistance
import calendar
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [14]:
actionability_train = df = pd.read_csv('~/w210_imwithdata/data/static_data/full_rzst_test_dataset.csv')

In [3]:
# import sys  

# reload(sys)  
# sys.setdefaultencoding('utf8')

In [15]:
actionability_train[:10]

,issue,action,id,es_score,tweet_timestamp,query_timestamp,tweet_user,tweet,actionable
0,healthcare,charity,AVrp-o7QMNAj8foZ-YaR,8.655990,2017-03-20T04:24:54.000Z,2017-03-19T23:37:51.916551,AnwaraliAnwaar,https://t.co/9vAellIc6i - RSS Channel: Colin K...,0
1,healthcare,charity,AVrpI7X7MNAj8foZ-Sjm,7.009409,2017-03-20T00:30:14.000Z,2017-03-19T23:37:51.916551,CrowdExposure,Please Support @kevinsanders23 Baby Bombers an...,1
2,healthcare,charity,AVrpM21hMNAj8foZ-TFR,7.009409,2017-03-20T00:47:24.000Z,2017-03-19T23:37:51.916551,KimothyJoy,Reminder that the GOP alternative to the Affor...,1
3,healthcare,charity,AVrpDUi3MNAj8foZ-Rx5,6.756997,2017-03-20T00:05:45.000Z,2017-03-19T23:37:51.916551,RussWhitworth,Don't forget to donate accordingly to https://...,1
4,healthcare,charity,AVrpsYiZMNAj8foZ-Ws5,6.756997,2017-03-20T03:05:08.000Z,2017-03-19T23:37:51.916551,TheRealLilXza,I'm raising money for ACA- Affordable Care Act...,1
5,healthcare,charity,AVrqSJjVMNAj8foZ-ZrD,6.756997,2017-03-20T05:50:08.000Z,2017-03-19T23:37:51.916551,Clearlease,Please Support @kevinsanders23 Baby Bombers an...,1
6,healthcare,charity,AVrpRRd9MNAj8foZ-Tkx,6.750567,2017-03-20T01:06:41.000Z,2017-03-19T23:37:51.916551,PawWorks,ALERT: Violet needs help!\r\rShe is at VCA Wes...,1
7,healthcare,charity,AVrpSkdTMNAj8foZ-Ttg,6.750567,2017-03-20T01:12:22.000Z,2017-03-19T23:37:51.916551,WineGumLady,@ConleyU @StephenKing we can't control who th...,0
8,healthcare,charity,AVrpVvedMNAj8foZ-UFu,6.750567,2017-03-20T01:26:13.000Z,2017-03-19T23:37:51.916551,JelliDonut,I will donate to your opposition. https://t.co...,0
9,healthcare,charity,AVrqBKmcMNAj8foZ-YmU,6.750567,2017-03-20T04:35:56.000Z,2017-03-19T23:37:51.916551,griffin_ghia,Say NO to Trumpcare and YES to the Affordable ...,1


In [16]:
tweet_text_list = actionability_train['tweet'].tolist()
tweet_text_list[:10]


['https://t.co/9vAellIc6i - RSS Channel: Colin Kaepernick pledges to donate money fromjersey sales https://t.co/mFnw8q7tyl',
 'Please Support @kevinsanders23 Baby Bombers and Medicare Donate Here - https://t.co/HLliHGEpNF ID=4',
 'Reminder that the GOP alternative to the Affordable Care Act defunds @ppfa _\xd9\xf7_Donate now and show\x89\xdb_ https://t.co/3aWSX3tXBb',
 "Don't forget to donate accordingly to https://t.co/pSpQmWIGV1. The AARP spends more than $7 million/yr lobbying fed gvt. to protect Medicare",
 "I'm raising money for ACA- Affordable Care Act Fund. Click to Donate:  https://t.co/9ymuQZLZbc via @gofundme",
 'Please Support @kevinsanders23 Baby Bombers and Medicare Donate Here - https://t.co/szNPbaCXIT ID=5',
 'ALERT: Violet needs help!\r\rShe is at VCA Westlake now with a prolapsed rectum for the third time\r\rDonate at\x89\xdb_ https://t.co/0BYOk23LkA',
 "@ConleyU  @StephenKing we can't control who they choose to donate to. should people/kids die/go hungry if they didn'

In [17]:
all_tweet_phones = []

for tweet in tweet_text_list:
    matches = []
    if phonenumbers.PhoneNumberMatcher(tweet.decode('latin-1'), "US"):
        for match in phonenumbers.PhoneNumberMatcher(tweet.decode('latin-1'), "US"):
            matches.append(phonenumbers.format_number(match.number, phonenumbers.PhoneNumberFormat.NATIONAL))
        all_tweet_phones.append(matches)
    else:
        all_tweet_phones.append([])


print all_tweet_phones[:1000]
        

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [18]:
all_tweet_urls = []

# ANY_URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
WEB_URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

for tweet in tweet_text_list:
    matches = []
    if re.findall(WEB_URL_REGEX,tweet):
        all_tweet_urls.append(re.findall(WEB_URL_REGEX,tweet))
    else:
        all_tweet_urls.append([])
        



In [19]:
all_tweet_urls[:10]

[['https://t.co/9vAellIc6i', 'https://t.co/mFnw8q7tyl'],
 ['https://t.co/HLliHGEpNF'],
 ['https://t.co/3aWSX3tXBb'],
 ['https://t.co/pSpQmWIGV1'],
 ['https://t.co/9ymuQZLZbc'],
 ['https://t.co/szNPbaCXIT'],
 ['https://t.co/0BYOk23LkA'],
 [],
 ['https://t.co/01w9JjVg23'],
 ['https://t.co/rJraok6mOf']]

In [ ]:
# all_tweet_city_state = []

# for tweet in tweet_text_list:
#     try:
#         places = geograpy.get_place_context(text = tweet)
#         if places.city_mentions:
#             print places.city_mentions
#         if places.region_mentions:
#             print places.region_mentions
#     except:
#         pass
#     else:
#         all_tweet_urls.append([])
        

In [ ]:
# geolocator = Nominatim()


In [20]:
# for urls in all_tweet_urls:
#     if urls:
#         try:
#             places = geograpy.get_place_context(url = urls[0])
#             if places.city_mentions:
#                 print places.city_mentions
#             if places.region_mentions:
#                 print places.region_mentions
#         except:
#             pass

In [12]:
# print os.environ.get('STANFORD_MODELS','/etc/environment')

# # Output:
# #/home/jackspicer/StanfordParser/stanford-ner-2015-12-09/classifiers/english.all.3class.distsim.crf.ser.gz:/home/jackspicer/stanford-postagger-2015-12-09/models/english.conll.4class.distsim.crf.ser.gz:/home/jackspicer/StanfordParser/stanford-ner-2015-12-09/english.muc.7class.distsim.crf.ser.gz

# # I want to select the first classifier model

# stanford_classifier = os.environ.get('STANFORD_MODELS','/etc/environment').split(':')[0]

# # For getting the path for StanfordNERTagger 
# stanford_ner_path = os.environ.get('CLASSPATH','/etc/environment').split(':')[0]

# st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')

# print st.tag("Manash is awesome!".split())

/etc/environment
Error: Could not find or load main class edu.stanford.nlp.ie.crf.CRFClassifier



OSError: Java command failed : [u'/usr/bin/java', '-mx1000m', '-cp', '', 'edu.stanford.nlp.ie.crf.CRFClassifier', '-loadClassifier', '/etc/environment', '-textFile', '/tmp/tmpFA99r3', '-outputFormat', 'slashTags', '-tokenizerFactory', 'edu.stanford.nlp.process.WhitespaceTokenizer', '-tokenizerOptions', '"tokenizeNLs=false"', '-encoding', 'utf-8']

In [21]:
tweet_text_list[0]

'https://t.co/9vAellIc6i - RSS Channel: Colin Kaepernick pledges to donate money fromjersey sales https://t.co/mFnw8q7tyl'

In [22]:
tweet_text_list[1]

'Please Support @kevinsanders23 Baby Bombers and Medicare Donate Here - https://t.co/HLliHGEpNF ID=4'

In [30]:
int(editdistance.eval(tweet_text_list[0],tweet_text_list[1]))

90

In [39]:
sorted([3,1,2])

[1, 2, 3]

In [46]:
distance_dict = {}

for i, tweet in enumerate(tweet_text_list[:1000]):
    for j, tweet2 in enumerate(tweet_text_list[:1000]):
        tweet_ids = tuple(sorted([i,j]))
        distance_dict[tweet_ids] = int(editdistance.eval(tweet,tweet2))

In [52]:
print take(100, distance_dict.iteritems())

[((668, 732), 109), ((381, 392), 88), ((703, 837), 102), ((259, 938), 119), ((432, 939), 107), ((150, 805), 117), ((337, 866), 107), ((247, 483), 120), ((284, 367), 113), ((747, 795), 119), ((648, 678), 106), ((0, 862), 117), ((203, 889), 119), ((613, 958), 113), ((25, 701), 99), ((544, 667), 122), ((228, 412), 120), ((122, 635), 116), ((69, 912), 112), ((703, 896), 109), ((290, 555), 113), ((94, 509), 104), ((424, 521), 101), ((209, 569), 118), ((262, 589), 109), ((156, 192), 95), ((359, 971), 100), ((234, 924), 115), ((181, 691), 114), ((75, 400), 120), ((22, 761), 49), ((367, 853), 116), ((522, 721), 119), ((100, 805), 107), ((619, 735), 117), ((47, 798), 116), ((453, 982), 104), ((56, 323), 108), ((251, 961), 115), ((296, 338), 84), ((499, 877), 116), ((812, 906), 121), ((650, 869), 87), ((134, 666), 96), ((28, 185), 93), ((321, 929), 101), ((493, 959), 120), ((268, 744), 89), ((53, 538), 101), ((346, 532), 120), ((581, 883), 110), ((187, 536), 115), ((553, 879), 100), ((78, 967), 

In [60]:
all_tweet_dates = []

for tweet in tweet_text_list:
    matches = []
    try:
        if dparser.parse(tweet,fuzzy=False):
            all_tweet_dates.append(dparser.parse(tweet,fuzzy=False))
        else:
            all_tweet_dates.append([])
    except:
        all_tweet_dates.append([])
        

In [75]:
# print all_tweet_dates[2000:5000]
# print tweet_text_list[:100]
print ('|').join(calendar.month_name[1:13])
print ('|').join(calendar.month_abbr[1:13])


January|February|March|April|May|June|July|August|September|October|November|December
Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec


In [107]:
all_tweet_dates = []

for tweet in tweet_text_list:
    matches = []
    if re.findall(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$', tweet):
        matches.append(re.findall(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$', tweet))
    if re.findall(r'(\d{2})[/.-](\d{2})$', tweet):
        matches.append(re.findall(r'(\d{2})[/.-](\d{2})$', tweet))
    if re.findall(r'\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\d{2}\d{4}', tweet):
        matches.append(re.findall(r'\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\d{2}\d{4}', tweet))
    if re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30), *((?:19|20)\d\d))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]), *((?:19|20)\d\d))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]), *((?:19|20)\d\d))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet):
        matches.append(re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30), *((?:19|20)\d\d))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]), *((?:19|20)\d\d))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]), *((?:19|20)\d\d))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet))
    elif re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet):
        matches.append(re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet))
    if re.findall(r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4}',tweet):
        matches.append(re.findall(r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4}',tweet))
    if re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}',tweet):
        matches.append(re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}',tweet))
    if re.findall(r'[0-9]{2}/[0-9]{2}',tweet):
        matches.append(re.findall(r'[0-9]{2}/[0-9]{2}',tweet))
    if re.findall(r'[0-9]{2}/[0-9]{1}',tweet):
        matches.append(re.findall(r'[0-9]{2}/[0-9]{1}',tweet))
#     if re.findall(r'\w{3,9}?\s\d{1,2}?\s,\s\d{4}?',tweet):
#         matches.append(re.findall(r'\w{3,9}?\s\d{1,2}?\s,\s\d{4}?',tweet))
    all_tweet_dates.append(matches)
        

# re.match(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$', tweet)
# re.match(r'(\d{2})[/.-](\d{2})$', tweet)
# re.match(r'\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}', text)
# re.match(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30), *((?:19|20)\d\d))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]), *((?:19|20)\d\d))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]), *((?:19|20)\d\d))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet)
# re.match(r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4}',tweet)
# re.match(r'^[a-zA-Z]{3}\s+\d{1,2}\s',tweet)
# re.match(r'\w{3,9}?\s\d{1,2}?\s,\s\d{4}?',tweet)


In [108]:
for i,dated in enumerate(all_tweet_dates):
    if len(dated) > 0:
        print i, ' ', dated, tweet_text_list[i]

1730   [[('', '', 'March', '2', '', '', '', '')]] Health Care Call to Action - March 20 to 24 https://t.co/juAxbq9yKh
2193   [[('', '', 'March', '2', '', '', '', '')]] Health Care Call to Action - March 20 to 24 - Help Save Health Care for Millions of Americans - Information on... https://t.co/OPJ9IvwJM9
2337   [['11/6']] #doyourjob @SenateFloor @SenatorTester @SenateGOP @HouseFloor @HouseGOP #voteno or #wevote 11/6/18 it won't be pret��_ https://t.co/f9xX3MWPkZ
2808   [[('', '', 'March', '2', '', '', '', '')]] Join us on March 23 for a CPR healthcare class. You could help save a life! Email thorn@sturgishospital.com to... https://t.co/9Ri1Xf31dL
3423   [[('', '', 'January', '2', '', '', '', '')]] #HR676 Healthcare For All Act has been in Congress since January 24th Call your Representatives to support��_ https://t.co/iWRnrZ1325
4389   [[('', '', 'January', '2', '', '', '', '')]] @SenTedCruz 4 questions about Obama Care , play Jonathan Gruber clip from early January 2017. ACA was never